In [1]:
import sys
sys.path.append('..')

In [2]:
import sqlite3
import pandas as pd
from copy import deepcopy
from os.path import join
import gc

In [3]:
flood_conn = sqlite3.connect('D:/GDrive/DATA/Pedro/Professional/Consulting/2021/01-01 - Network criticality analysis (UArk)/DATA/flood_layer.sqlite')
flooded = pd.read_sql('Select link_id from links', flood_conn)

In [4]:
from os.path import join
from aequilibrae import Project
from aequilibrae.paths import TrafficAssignment, TrafficClass
import logging
import sys

import numpy as np

In [5]:
pth = 'D:/GDrive/DATA/Pedro/Professional/Consulting/2021/01-01 - Network criticality analysis (UArk)/DATA/simple_model'

proj = Project()
proj.open(pth)
net = proj.network

In [6]:
%%time
proj_matrices = proj.matrices
mat_list = proj_matrices.list()

modes = net.modes
modes.all_modes()

nodes = net.nodes

for n in [4701, 4702, 4703]:
    nd = nodes.get(n)
    nd.is_centroid = 0
    nd.save()

net.build_graphs(modes=['T'])
truckGraph = net.graphs['T']
net.build_graphs(modes=['c'])
carGraph = net.graphs['c']

Wall time: 1min


In [7]:
%%time
# Link exclusions

exclude_from_passenger = [x[0] for x in proj.conn.execute("select link_id from links where exclusionset IN ('TruckOnly', 'HOV2', 'HOV3')")]

exclude_from_truck = [x[0] for x in proj.conn.execute("select link_id from links where exclusionset IN ('PassengerOnly', 'HOV2', 'HOV3')")]


graph = truckGraph
set1 = graph.network[(graph.network.builtyear > 2010) | (graph.network.removedyear < 2010)].link_id.to_list()
set2 = graph.network[(graph.network.mode_code <10) | (graph.network.mode_code >11)].link_id.to_list()
exclude_from_passenger.extend(set1 + set2)
exclude_from_truck.extend(set1 + set2)

truckGraph.exclude_links(exclude_from_truck)
carGraph.exclude_links(exclude_from_passenger)

nodes = net.nodes
for n in [4701, 4702, 4703]:
    nd = nodes.get(n)
    nd.is_centroid = 1
    nd.save()

for graph in [carGraph, truckGraph]:
    graph.graph.alpha.fillna(0.15, inplace=True)
    graph.graph.beta.fillna(4.0, inplace=True)
    graph.graph.hov1tollcost.fillna(0, inplace=True)
    graph.graph.mttollcost.fillna(0, inplace=True)
    graph.graph.httollcost.fillna(0, inplace=True)
    for period in ['am', 'pm', 'md', 'nt']:
        graph.graph.loc[graph.graph.a_node == graph.graph.b_node, f"{period}_assncap_10"] = 1.0
        graph.graph.loc[graph.graph.a_node == graph.graph.b_node, f"tt_{period}_10"] = 0.001

Wall time: 16.1 s


## Baseline assignment

In [8]:
period= 'am'
has_baseline =  sum([x[0] for x in proj.conn.execute(f"select count(*) from results where table_name='{period}_baseline'")])
ref_carGraph = deepcopy(carGraph)
ref_truckGraph = deepcopy(truckGraph)
    
if has_baseline:
    conn = sqlite3.connect(join(pth, 'results_database.sqlite'))
    
    baseline = pd.read_sql(f"select * from '{period}_baseline'", conn)
    conn.close()
    pass
else:
    proj_matrices = proj.matrices
    carDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
    carDemand.computational_view('AUTO')

    lightTruckDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
    lightTruckDemand.computational_view('COMMTRK')
    lightTruckDemand.matrix_view = np.array(lightTruckDemand.matrix_view, np.float64)

    heavyTruckDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
    heavyTruckDemand.computational_view('HTRK')
    heavyTruckDemand.matrix_view = np.array(heavyTruckDemand.matrix_view, np.float64)

    assig = TrafficAssignment()
    # assig.procedure_id = f'{period}_baseline'
    carClass = TrafficClass('car', carGraph, carDemand)
    carClass.set_pce(1)
    carClass.set_vot(0.2)
    carClass.set_fixed_cost('hov1tollcost')

    # The link exclusions for commercial trucks are actually the same as the ones for passenger cars, and not heavy trucks
    lightTruckClass = TrafficClass('light_truck', carGraph, lightTruckDemand)
    lightTruckClass.set_pce(1.5)
    lightTruckClass.set_vot(0.5)
    lightTruckClass.set_fixed_cost("mttollcost")

    heavyTruckClass = TrafficClass('heavy_truck', truckGraph, heavyTruckDemand)
    heavyTruckClass.set_pce(2.5)
    heavyTruckClass.set_vot(1.0)
    heavyTruckClass.set_fixed_cost("httollcost")

    # The first thing to do is to add at list of traffic classes to be assigned
    assig.set_classes([heavyTruckClass, carClass, lightTruckClass])
    assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

    assig.set_vdf_parameters({"alpha": "alpha", "beta": "beta"})  # And its parameters

    assig.set_time_field(f"tt_{period}_10")
    assig.set_capacity_field(f"{period}_assncap_10")  # The capacity and free flow travel times as they exist in the graph

    # And the algorithm we want to use to assign
    assig.set_algorithm('bfw')
    assig.max_iter = 100
    assig.rgap_target = 0.001

    assig.execute()  # we then execute the assignment
    assig.save_results(f'{period}_baseline')
    baseline = assig.results()



### Loop

In [9]:
proj_matrices = proj.matrices
carDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
carDemand.computational_view('AUTO')

lightTruckDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
lightTruckDemand.computational_view('COMMTRK')
lightTruckDemand.matrix_view = np.array(lightTruckDemand.matrix_view, np.float64)

heavyTruckDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
heavyTruckDemand.computational_view('HTRK')
heavyTruckDemand.matrix_view = np.array(heavyTruckDemand.matrix_view, np.float64)

    
def assign_am(proj, scenario_name, carGraph, truckGraph):
    period= 'am'
    
    assig = TrafficAssignment()
#     assig.procedure_id = scenario_name
    carClass = TrafficClass('car', carGraph, carDemand)
    carClass.set_pce(1)
    carClass.set_vot(0.2)
    carClass.set_fixed_cost('hov1tollcost')

    # The link exclusions for commercial trucks are actually the same as the ones for passenger cars, and not heavy trucks
    lightTruckClass = TrafficClass('light_truck', carGraph, lightTruckDemand)
    lightTruckClass.set_pce(1.5)
    lightTruckClass.set_vot(0.5)
    lightTruckClass.set_fixed_cost("mttollcost")

    heavyTruckClass = TrafficClass('heavy_truck', truckGraph, heavyTruckDemand)
    heavyTruckClass.set_pce(2.5)
    heavyTruckClass.set_vot(1.0)
    heavyTruckClass.set_fixed_cost("httollcost")

    # The first thing to do is to add at list of traffic classes to be assigned
    assig.set_classes([heavyTruckClass, carClass, lightTruckClass])
    assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

    assig.set_vdf_parameters({"alpha": "alpha", "beta": "beta"})  # And its parameters

    assig.set_time_field(f"tt_{period}_10")
    assig.set_capacity_field(f"{period}_assncap_10")  # The capacity and free flow travel times as they exist in the graph

    # And the algorithm we want to use to assign
    assig.set_algorithm('bfw')
    assig.max_iter = 100
    assig.rgap_target = 0.001

    assig.execute()  # we then execute the assignment
    assig.save_results(scenario_name)
    for item in [lightTruckClass, carClass, heavyTruckClass, assig]:
        del item


In [10]:
compressed_ids = ref_carGraph.graph[['link_id', '__compressed_id__']]
compressed = compressed_ids.merge(flooded, on='link_id')
compressed_unique = compressed.__compressed_id__.unique()

In [11]:
compressed_unique = sorted(list(compressed_unique))

In [ ]:
used_compressed_ids = []
cancelled_physical_links = []
counter = 0
for clink in compressed_unique:
    gc.collect()
    print(f'Analyzing link_id {clink}')
    if clink in used_compressed_ids:
        print('    Link already considered')
        continue
    counter += 1
    
    
    scenario_name = f'exclude_link_id_{clink}'
    
    if sum([x[0] for x in proj.conn.execute(f'select count(*) from results where table_name = "{scenario_name}"')]) > 0:
        print('    Scenario already computed')
        continue
    
    carGraph = deepcopy(ref_carGraph)
    truckGraph = deepcopy(ref_truckGraph)
    
    for graph in [carGraph, truckGraph]:
        # Figure out who to remove and add to list
        link_ids = graph.graph.loc[graph.graph.__compressed_id__== clink, 'link_id'].tolist()
        cancelled_hyper_links = graph.graph.loc[graph.graph.link_id.isin(link_ids), '__compressed_id__'].unique().tolist()
        used_compressed_ids.extend(cancelled_hyper_links)
        cancelled_physical_links.extend(link_ids)

        graph.network.loc[graph.network.link_id.isin(link_ids), 'a_node'] = graph.network.loc[graph.network.link_id.isin(link_ids), 'b_node']
    
    baseline_flow = np.sum(baseline.loc[baseline.link_id==link_ids[0],'PCE_tot'])
    if baseline_flow == 0:
        print(f'   Link {clink} already has zero flow in the baseline')
        continue
    
    for graph in [carGraph, truckGraph]:
        graph.prepare_graph(graph.centroids)
        
        graph.graph.alpha.fillna(0.15, inplace=True)
        graph.graph.beta.fillna(4.0, inplace=True)
        graph.graph.hov1tollcost.fillna(0, inplace=True)
        graph.graph.mttollcost.fillna(0, inplace=True)
        graph.graph.httollcost.fillna(0, inplace=True)
        for period in ['am', 'pm', 'md', 'nt']:
            graph.graph.loc[graph.graph.a_node == graph.graph.b_node, f"{period}_assncap_10"] = 1.0
            graph.graph.loc[graph.graph.a_node == graph.graph.b_node, f"tt_{period}_10"] = 0.001
    
    
    print(f'   Link {clink} has {round(baseline_flow, 1)} PCE flow in the baseline')
    assign_am(proj, scenario_name, carGraph, truckGraph)
    for graph in [carGraph, truckGraph]:
        del graph
    

Analyzing link_id 35.0
    Scenario already computed
Analyzing link_id 3805.0
    Scenario already computed
Analyzing link_id 4488.0
    Scenario already computed
Analyzing link_id 6574.0
    Scenario already computed
Analyzing link_id 8543.0
    Scenario already computed
Analyzing link_id 8546.0
    Scenario already computed
Analyzing link_id 15550.0
    Scenario already computed
Analyzing link_id 15551.0
    Scenario already computed
Analyzing link_id 15553.0
    Scenario already computed
Analyzing link_id 17568.0
    Scenario already computed
Analyzing link_id 17569.0
    Scenario already computed
Analyzing link_id 17570.0
    Scenario already computed
Analyzing link_id 17819.0
    Scenario already computed
Analyzing link_id 17820.0
    Scenario already computed
Analyzing link_id 17830.0
    Scenario already computed
Analyzing link_id 17918.0
    Scenario already computed
Analyzing link_id 17977.0
    Scenario already computed
Analyzing link_id 18298.0
    Scenario already computed


Analyzing link_id 25647.0
    Scenario already computed
Analyzing link_id 25648.0
    Scenario already computed
Analyzing link_id 25649.0
    Scenario already computed
Analyzing link_id 25652.0
    Scenario already computed
Analyzing link_id 25696.0
    Scenario already computed
Analyzing link_id 25697.0
    Scenario already computed
Analyzing link_id 25698.0
    Scenario already computed
Analyzing link_id 25699.0
    Scenario already computed
Analyzing link_id 25705.0
    Scenario already computed
Analyzing link_id 25714.0
    Scenario already computed
Analyzing link_id 25716.0
    Scenario already computed
Analyzing link_id 25717.0
    Scenario already computed
Analyzing link_id 25718.0
    Scenario already computed
Analyzing link_id 25721.0
    Scenario already computed
Analyzing link_id 25722.0
    Scenario already computed
Analyzing link_id 25727.0
    Scenario already computed
Analyzing link_id 25728.0
    Scenario already computed
Analyzing link_id 25729.0
    Scenario already c

   Link 29918.0 already has zero flow in the baseline
Analyzing link_id 29920.0
    Link already considered
Analyzing link_id 29921.0
    Scenario already computed
Analyzing link_id 29925.0
    Scenario already computed
Analyzing link_id 29927.0
    Scenario already computed
Analyzing link_id 29928.0
    Scenario already computed
Analyzing link_id 29930.0
    Scenario already computed
Analyzing link_id 29941.0
    Scenario already computed
Analyzing link_id 29942.0
    Scenario already computed
Analyzing link_id 29945.0
    Scenario already computed
Analyzing link_id 29946.0
    Scenario already computed
Analyzing link_id 30030.0
    Scenario already computed
Analyzing link_id 30031.0
    Scenario already computed
Analyzing link_id 31313.0
    Scenario already computed
Analyzing link_id 31416.0
    Scenario already computed
Analyzing link_id 31436.0
    Scenario already computed
Analyzing link_id 31437.0
    Scenario already computed
Analyzing link_id 31609.0
    Scenario already compu

Analyzing link_id 35367.0
    Scenario already computed
Analyzing link_id 35368.0
    Scenario already computed
Analyzing link_id 35369.0
    Scenario already computed
Analyzing link_id 35371.0
    Scenario already computed
Analyzing link_id 35595.0
    Scenario already computed
Analyzing link_id 35597.0
    Scenario already computed
Analyzing link_id 35602.0
   Link 35602.0 already has zero flow in the baseline
Analyzing link_id 35603.0
   Link 35603.0 already has zero flow in the baseline
Analyzing link_id 35610.0
    Link already considered
Analyzing link_id 35612.0
    Scenario already computed
Analyzing link_id 35614.0
    Scenario already computed
Analyzing link_id 35615.0
    Scenario already computed
Analyzing link_id 35715.0
    Scenario already computed
Analyzing link_id 35716.0
    Scenario already computed
Analyzing link_id 35717.0
    Scenario already computed
Analyzing link_id 35721.0
    Scenario already computed
Analyzing link_id 35722.0
    Scenario already computed
An

Analyzing link_id 44220.0
    Scenario already computed
Analyzing link_id 44274.0
    Scenario already computed
Analyzing link_id 44275.0
    Scenario already computed
Analyzing link_id 44276.0
    Scenario already computed
Analyzing link_id 44277.0
    Scenario already computed
Analyzing link_id 44279.0
    Scenario already computed
Analyzing link_id 44280.0
    Scenario already computed
Analyzing link_id 44282.0
    Scenario already computed
Analyzing link_id 44283.0
    Scenario already computed
Analyzing link_id 44288.0
    Scenario already computed
Analyzing link_id 44289.0
    Scenario already computed
Analyzing link_id 44301.0
    Scenario already computed
Analyzing link_id 44312.0
    Scenario already computed
Analyzing link_id 44315.0
    Scenario already computed
Analyzing link_id 44317.0
    Scenario already computed
Analyzing link_id 44319.0
    Scenario already computed
Analyzing link_id 44320.0
    Scenario already computed
Analyzing link_id 44323.0
    Scenario already c

Analyzing link_id 46149.0
    Scenario already computed
Analyzing link_id 46173.0
    Scenario already computed
Analyzing link_id 46174.0
    Scenario already computed
Analyzing link_id 46176.0
    Scenario already computed
Analyzing link_id 46177.0
    Scenario already computed
Analyzing link_id 46183.0
    Scenario already computed
Analyzing link_id 46186.0
   Link 46186.0 has 923.5 PCE flow in the baseline
Analyzing link_id 46187.0
    Scenario already computed
Analyzing link_id 46189.0
    Scenario already computed
Analyzing link_id 46190.0
   Link 46190.0 has 864.5 PCE flow in the baseline
Analyzing link_id 46191.0
    Scenario already computed
Analyzing link_id 46193.0
    Scenario already computed
Analyzing link_id 46194.0
    Scenario already computed
Analyzing link_id 46196.0
    Scenario already computed
Analyzing link_id 46199.0
    Scenario already computed
Analyzing link_id 46200.0
   Link 46200.0 has 313.7 PCE flow in the baseline
Analyzing link_id 46201.0
    Scenario al

   Link 46823.0 has 25.1 PCE flow in the baseline
Analyzing link_id 46829.0
    Scenario already computed
Analyzing link_id 46830.0
    Scenario already computed
Analyzing link_id 46832.0
   Link 46832.0 has 106.1 PCE flow in the baseline
Analyzing link_id 46835.0
   Link 46835.0 has 41.7 PCE flow in the baseline
Analyzing link_id 46836.0
   Link 46836.0 has 18.3 PCE flow in the baseline
Analyzing link_id 46841.0
   Link 46841.0 has 56.0 PCE flow in the baseline
Analyzing link_id 46842.0
    Scenario already computed
Analyzing link_id 46844.0
   Link 46844.0 has 139.6 PCE flow in the baseline
Analyzing link_id 46845.0
   Link 46845.0 has 136.7 PCE flow in the baseline
Analyzing link_id 46847.0
    Scenario already computed
Analyzing link_id 46848.0
    Scenario already computed
Analyzing link_id 46853.0
    Scenario already computed
Analyzing link_id 46856.0
   Link 46856.0 has 210.7 PCE flow in the baseline
Analyzing link_id 46857.0
   Link 46857.0 has 68.9 PCE flow in the baseline
An

Analyzing link_id 48088.0
    Scenario already computed
Analyzing link_id 48100.0
   Link 48100.0 has 61.3 PCE flow in the baseline
Analyzing link_id 48107.0
    Scenario already computed
Analyzing link_id 48108.0
    Scenario already computed
Analyzing link_id 48111.0
   Link 48111.0 has 79.1 PCE flow in the baseline
Analyzing link_id 48113.0
   Link 48113.0 has 42.8 PCE flow in the baseline
Analyzing link_id 48114.0
   Link 48114.0 has 42.8 PCE flow in the baseline
Analyzing link_id 48116.0
   Link 48116.0 has 22.5 PCE flow in the baseline
Analyzing link_id 48141.0
    Scenario already computed
Analyzing link_id 48143.0
   Link 48143.0 has 12.9 PCE flow in the baseline
Analyzing link_id 48144.0
   Link 48144.0 has 258.0 PCE flow in the baseline
Analyzing link_id 48146.0
    Scenario already computed
Analyzing link_id 48153.0
    Scenario already computed
Analyzing link_id 48166.0
    Scenario already computed
Analyzing link_id 48167.0
    Scenario already computed
Analyzing link_id 4

In [ ]:
np.diag(carDemand.matrix_view[:,:,0])